<a href="https://colab.research.google.com/github/kxaqw/QR_decomposition_Householder/blob/main/sharapova_qr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import math

def sign(z): #Safe sign function
    return 1 if z >= 0 else -1

def vector_norm(x): #Compute the 2-norm of a vector or matrix (Frobenius norm)
    if len(x.shape) == 1:  # For vectors
        sum_sq = 0.0
        for xi in x:
            sum_sq += xi * xi
        return math.sqrt(sum_sq)
    else:  # For matrices
        sum_sq = 0.0
        for row in x:
            for elem in row:
                sum_sq += elem * elem
        return math.sqrt(sum_sq)

def dot_product(x, y): #Compute dot product of two vectors
    return sum(xi * yi for xi, yi in zip(x, y))

def outer_product(x, y):#Compute outer product of two vectors
    m = len(x)
    n = len(y)
    result = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            result[i,j] = x[i] * y[j]
    return result

def matrix_multiply(A, B): #Matrix multiplication using basic operations
    m, n = A.shape
    p = B.shape[1]
    result = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            for k in range(n):
                result[i,j] += A[i,k] * B[k,j]
    return result

def round_matrix(M, decimals=3): #Round matrix elements to specified decimal places
    return np.round(M, decimals)

def is_upper_triangular(R, tol=1e-10): #Check if matrix is upper triangular
    m, n = R.shape
    for i in range(1, m):
        for j in range(0, min(i, n)):
            if abs(R[i,j]) > tol:
                return False
    return True

def householder_qr(A):
    """
    Compute QR decomposition of matrix A using Householder reflections.
    Returns rounded matrices Q and R, and list of all H matrices.
    """
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy().astype(float)
    H_matrices = []  # To store all H matrices
    intermediate_results = []  # To store Hk*A after each step

    for k in range(min(m, n)):
        x = R[k:, k]
        norm_x = vector_norm(x)

        if norm_x < 1e-12:
            continue

        e1 = np.zeros_like(x)
        e1[0] = 1

        v = sign(x[0]) * norm_x * e1 + x
        v_norm = vector_norm(v)
        if v_norm < 1e-12:
            continue
        v = v / v_norm

        H_k = np.eye(m)
        v_outer = outer_product(v, v)
        H_k_sub = np.eye(len(x)) - 2 * v_outer
        H_k[k:, k:] = H_k_sub

        R = matrix_multiply(H_k, R)
        Q = matrix_multiply(Q, H_k.T)
        H_matrices.append(H_k.copy())  # Save the H matrix
        intermediate_results.append(R.copy())  # Save intermediate R

    # Round the results to 3 decimal places for output
    Q_rounded = round_matrix(Q)
    R_rounded = round_matrix(R)
    H_matrices_rounded = [round_matrix(H) for H in H_matrices]

    return Q, R, Q_rounded, R_rounded, H_matrices_rounded, intermediate_results

def read_matrix(filename):
    """Read matrix from file"""
    with open(filename, 'r') as f:
        first_line = f.readline().strip()
        m, n = map(int, first_line.split())
        data = []
        for _ in range(m):
            line = f.readline().strip()
            row = list(map(float, line.split()))
            data.append(row)
    return np.array(data)

def write_matrix(filename, matrix):
    """Write matrix to file with 3 decimal places"""
    with open(filename, 'w') as f:
        for row in matrix:
            line = ' '.join([f'{x:.3f}' for x in row])
            f.write(line + '\n')

def verify_decomposition(A, Q, R): #Verify the QR decomposition with detailed checks
    QR = matrix_multiply(Q, R)

    print("\nDetailed Verification:")
    print("1. Checking A = QR:")
    error_QR = vector_norm(A - QR)
    print(f"||A - QR|| = {error_QR:.3e}")

    print("\n2. Checking orthogonality of Q:")
    QtQ = matrix_multiply(Q.T, Q)
    error_ortho = vector_norm(QtQ - np.eye(Q.shape[0]))
    print(f"||QᵀQ - I|| = {error_ortho:.3e}")

    print("\n3. Checking R is upper triangular:")
    is_upper = is_upper_triangular(R)
    print(f"R is upper triangular: {is_upper}")

    print("\n4. Verification summary:")
    print(f"All checks passed: {error_QR < 1e-10 and error_ortho < 1e-10 and is_upper}")


def main():
    try:
        A = read_matrix('matrix_c.txt')
    except FileNotFoundError:
        print("Error: matrix_x.txt not found.")
        return

    Q, R, Q_rounded, R_rounded, H_matrices, intermediates = householder_qr(A)

    write_matrix('Q.txt', Q_rounded)
    write_matrix('R.txt', R_rounded)

    print("Final Q matrix (rounded):")
    print(Q_rounded)
    print("\nFinal R matrix (rounded):")
    print(R_rounded)

    verify_decomposition(A, Q, R)

if __name__ == "__main__":
    main()

Final Q matrix (rounded):
[[-0.408  0.123 -0.905]
 [-0.816 -0.492  0.302]
 [-0.408  0.862  0.302]]

Final R matrix (rounded):
[[-2.449 -2.041]
 [ 0.     1.354]
 [-0.     0.   ]]

Detailed Verification:
1. Checking A = QR:
||A - QR|| = 5.661e-16

2. Checking orthogonality of Q:
||QᵀQ - I|| = 2.748e-16

3. Checking R is upper triangular:
R is upper triangular: True

4. Verification summary:
All checks passed: True
